# Example of BOMD MD (NVE, NVT)

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/MD_BOMD_Basic'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   508.67   2.630030e-01 -1.350658e+00 -1.087655e+00 ||    355.80   1.839639e-01 1.501225e+00 1.685189e+00 || 
     2   470.57   2.433021e-01 -1.330798e+00 -1.087495e+00 ||    255.68   1.321945e-01 1.553362e+00 1.685556e+00 || 
     3   421.10   2.177281e-01 -1.305032e+00 -1.087304e+00 ||    198.27   1.025123e-01 1.583227e+00 1.685740e+00 || 
     4   366.20   1.893379e-01 -1.276449e+00 -1.087111e+00 ||    189.23   9.784064e-02 1.587896e+00 1.685737e+00 || 
     5   311.86   1.612447e-01 -1.248187e+00 -1.086942e+00 ||    228.49   1.181385e-01 1.567413e+00 1.685551e+00 || 
     6   263.41   1.361934e-01 -1.223013e+00 -1.086819e+00 ||    310.33   1.604555e-01 1.524734e+00 1.685190e+00 || 
     7   224.78   1.162215e-01 -1.202978e+00 -1.086757e+00 ||    423.77   2.191057e-01 1.465565e+00 1.684671e+00 || 
     8   198.12   1.024360e-01 -1.189196e+00 -1.086760e+00 ||    553.16   2.860066e-01 1.39801

## *MD_BOMD_BASIC.0.xyz* and *MD_BOMD_BASIC.1.xyz* files with general info for each step are created. Coordinates, velocities, forces and charges are recorded in the following sequence: x y z vx vy vz fx fy fz q.

## *MD_BOMD_BASIC.Info.0.txt* and *MD_BOMD_BASIC.Info.1.txt* contain additional info for each step (orbital energies, dipole).

## Scale velocities to adjust kinetic energy and compenstate the energy shift

In [4]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/MD_BOMD_Basic_E_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, control_energy_shift=True, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   207.34   1.072017e-01 -8.764901e-02 1.955273e-02 ||    572.13   2.958167e-01 2.073125e+00 2.368942e+00 || 
     2   243.73   1.260159e-01 -1.064632e-01 1.955273e-02 ||    769.46   3.978415e-01 1.971100e+00 2.368942e+00 || 
     3   327.00   1.690699e-01 -1.495172e-01 1.955273e-02 ||   1004.55   5.193913e-01 1.849550e+00 2.368942e+00 || 
     4   459.44   2.375472e-01 -2.179945e-01 1.955273e-02 ||   1255.65   6.492192e-01 1.719722e+00 2.368942e+00 || 
     5   643.40   3.326630e-01 -3.131103e-01 1.955273e-02 ||   1498.32   7.746891e-01 1.594253e+00 2.368942e+00 || 
     6   878.87   4.544116e-01 -4.348589e-01 1.955273e-02 ||   1706.79   8.824812e-01 1.486461e+00 2.368942e+00 || 
     7  1160.55   6.000496e-01 -5.804969e-01 1.955273e-02 ||   1854.96   9.590909e-01 1.409851e+00 2.368942e+00 || 
     8  1475.45   7.628657e-01 -7.433130e-01 1.955273e-02 ||   1917.08   9.912088e-01 1.377733e+00 2.

## Scale velocities to control temperature

In [5]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/MD_BOMD_Basic_T_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, scale_vel=[1,400], remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.00   2.068160e-01 -5.892348e-02 1.478925e-01 ||    400.00   2.068160e-01 2.143274e+00 2.350090e+00 || 
     2   400.00   2.068160e-01 -5.217212e-02 1.546439e-01 ||    400.00   2.068160e-01 2.097205e+00 2.304021e+00 || 
     3   400.00   2.068160e-01 -7.434265e-02 1.324733e-01 ||    400.00   2.068160e-01 2.018711e+00 2.225527e+00 || 
     4   400.00   2.068160e-01 -1.229255e-01 8.389051e-02 ||    400.00   2.068160e-01 1.919684e+00 2.126500e+00 || 
     5   400.00   2.068160e-01 -1.925878e-01 1.422819e-02 ||    400.00   2.068160e-01 1.811000e+00 2.017816e+00 || 
     6   400.00   2.068160e-01 -2.770935e-01 -7.027755e-02 ||    400.00   2.068160e-01 1.700871e+00 1.907687e+00 || 
     7   400.00   2.068160e-01 -3.706762e-01 -1.638602e-01 ||    400.00   2.068160e-01 1.594995e+00 1.801811e+00 || 
     8   400.00   2.068160e-01 -4.686285e-01 -2.618125e-01 ||    400.00   2.068160e-01 1.497239e+00

## Langevin Thermostat

In [6]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/MD_BOMD_Langevin'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# specify damping factor
md =  Molecular_Dynamics_Langevin(damp = 100.0, seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   393.63   2.035200e-01 -1.348049e+00 -1.144529e+00 ||    462.23   2.389917e-01 1.388268e+00 1.627260e+00 || 
     2   331.12   1.712012e-01 -1.324041e+00 -1.152840e+00 ||    550.69   2.847291e-01 1.340948e+00 1.625677e+00 || 
     3   277.40   1.434246e-01 -1.294431e+00 -1.151006e+00 ||    649.42   3.357736e-01 1.297567e+00 1.633340e+00 || 
     4   217.36   1.123852e-01 -1.259408e+00 -1.147023e+00 ||    713.64   3.689780e-01 1.264546e+00 1.633524e+00 || 
     5   145.39   7.517252e-02 -1.226805e+00 -1.151632e+00 ||    723.48   3.740661e-01 1.248836e+00 1.622902e+00 || 
     6   105.60   5.459880e-02 -1.199979e+00 -1.145380e+00 ||    690.98   3.572648e-01 1.258605e+00 1.615870e+00 || 
     7    64.71   3.345663e-02 -1.176330e+00 -1.142874e+00 ||    659.16   3.408096e-01 1.294412e+00 1.635222e+00 || 
     8    30.40   1.571696e-02 -1.162940e+00 -1.147223e+00 ||    581.69   3.007593e-01 1.34899